In [1]:
!pip install peft
!pip install Faiss-cpu
!pip install langchain

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from peft import PeftModel, PeftConfig
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.llms import OpenAIChat
from langchain.chat_models import ChatOpenAI

import torch
import re
import base64
import os

In [4]:
SEARCH_NUM = 3
conversation_pairs = []
loaded_memory =""
dialogues = ""
query2=""

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

In [6]:
pip install sentence-transformers

In [7]:
# csv load
from langchain.document_loaders import TextLoader

loader = TextLoader("/content/자동차_text.txt")
data=loader.load()

# text split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=0)
data_split = text_splitter.split_documents(data)

# embedding
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
db = FAISS.from_documents(data_split, embeddings)

docs_memory = []
cnt=0
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(그만):])).item():
                return True

        return False

In [8]:
# prompt template
template = '''
    너는 사실만을 말하는 금융 전문가야.
    질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 대답해.
    문서에서 질문에 대한 근거를 찾을 수 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 답해.
    문서 = {docs_input}
    질문 = {query_input}
'''

In [9]:
# prompt
prompt = PromptTemplate(
    input_variables=[
        "docs_input",
        "query_input",
    ],
    template=template,)

In [10]:
# search similarity
def db_search(query: str, k: int):
    docs = db.similarity_search(query, k)
    return docs

In [11]:
prompt

PromptTemplate(input_variables=['docs_input', 'query_input'], template='\n    너는 사실만을 말하는 금융 전문가야.\n    질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 대답해.\n    문서에서 질문에 대한 근거를 찾을 수 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 답해.\n    문서 = {docs_input}\n    질문 = {query_input}\n')

In [12]:
from langchain.chains import RetrievalQA

In [13]:
pip install -U torch transformers tokenizers accelerate

In [14]:
!pip install utils
!pip install utils.prompter

ERROR: Could not find a version that satisfies the requirement utils.prompter (from versions: none)
ERROR: No matching distribution found for utils.prompter


In [18]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# from utils.prompter import Prompter

# MODEL = "nlpai-lab/kullm-polyglot-5.8b-v2"

# model = AutoModelForCausalLM.from_pretrained(
#     MODEL,
#     torch_dtype=torch.float16,
#     low_cpu_mem_usage=True,
# ).to(device=f"cuda", non_blocking=True)
# model.eval()

# pipe = pipeline("text-generation", model=model, tokenizer=MODEL, device=0)

# prompter = Prompter("kullm")


# def infer(instruction="", input_text=""):
#     prompt = prompter.generate_prompt(instruction, input_text)
#     output = pipe(prompt, max_length=512, temperature=0.2, num_beams=5, eos_token_id=2)
#     s = output[0]["generated_text"]
#     result = prompter.get_response(s)

#     return result


# result = infer(input_text="현대차, 글로벌 도매 판매는 몇대를 기록했나요?")
# print(result)
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig

model='davidkim205/komt-mistral-7b-v1'
peft_model_name = 'davidkim205/komt-mistral-7b-v1-lora'
config = PeftConfig.from_pretrained(peft_model_name)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

config.base_model_name_or_path =model
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map="auto")
model = PeftModel.from_pretrained(model, peft_model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
!pip install sentencepiece

In [19]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
streamer = TextStreamer(tokenizer)

In [20]:
def load_chain(_prompt) :
    chain = LLMChain(
        llm=llm,
        prompt=prompt
    )
    return chain

In [21]:
retriever = db.as_retriever(search_kwargs={"k": 3})

In [22]:
#run chain
def run_chain(prompt, docs,query: str):
    chain = load_chain(prompt)
    response = chain.run(
    docs_input=docs,
    query_input=query,
    )
    return response

In [23]:
query='삼성전자의 3분기 영업이익은?'
docs = db_search(query, SEARCH_NUM)
docs = [doc.page_content for doc in docs]

In [24]:
docs

['• 내수 시장 판매, 4.33만 대(-2.6%MoM, +0.6%YoY) 및 M/S 36.9%(수입차 제외): 세단 1.31만 대(+15.4%MoM, -5.3%YoY)로,',
 '3,507대, 셀토스 4,355대로, RV 외 차종들의 판매 전년 대비 감소. 한편 2022년 2월 출시된 EV6의 판매는 1,542대(+30.0%YoY)로 다시 월 판매 2천',
 '• 고금리 영향 누적으로 자동차 수요에 대한 불안감 증폭. 4분기 현대차/기아의 연중 최대 생산 및 최대 판매로 우려 불식 예상. Value Chain에도 관심이 필요한 시기.']

In [25]:
def gen(x,docs_input):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=0.8,
        top_k=100,
        max_new_tokens=1024,
        early_stopping=True,
        do_sample=True,
    )
    template = f"""[INST]
        너는 사실만을 말하는 금융 전문가야.
        질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 대답해.
        문서 = {docs_input}
        질문 = {x} [/INST]
    """

    q = template
    # q = f"[INST]{x} [/INST]"
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

# result = gen('제주도를 1박2일로 혼자 여행하려고 하는데 여행 코스를 만들어줘')

# print('##########')
# print(result)

In [26]:
query='현대차, 글로벌 도매 판매는 몇대를 기록했나요?'
docs = db_search(query, SEARCH_NUM)
docs = [doc.page_content for doc in docs]
gen(query,docs)[-1]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


<s> [INST]
        너는 사실만을 말하는 금융 전문가야.
        질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 대답해.
        문서 = ['WHAT’S THE STORY?\n현대차, 글로벌 도매 판매 37.8만 대(+5.8%MoM, +9.6%YoY): 노사협상이 마무리되면서, 생산/판매 강한 회복 기록.', '• 현대차 미국 소매 판매, 66,529대(-11.8%MoM, +2.4%YoY) 및 M/S 5.5%: 주요 모델별로 팰리세이드 7,071대/ 싼타페 9,700대/ 투싼', '• 현대차/기아의 10월 글로벌 도매 판매는 각각 +9.6%YoY, +7.7%YoY 기록. 기아 판매가 현대차 대비 약한 것은 노사협상 지연으로 한국공장 특근 부재 영향']
        질문 = 현대차, 글로벌 도매 판매는 몇대를 기록했나요? [/INST]
    현대차, 글로벌 도매 판매는 37.8만 대를 기록했습니다.</s>


'>'

In [27]:
query='팰리세이드는 몇대 판매됐어??'
docs = db_search(query, SEARCH_NUM)
docs = [doc.page_content for doc in docs]
gen(query,docs)[-1]

<s> [INST]
        너는 사실만을 말하는 금융 전문가야.
        질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호하다면 내용을 찾을 수 없습니다. 라고 대답해.
        문서 = ['RV 2.87만 대(+21.7%MoM, +31.6%YoY) 판매, 대부분 차종의 판매가 전월대비 증가한 가운데 코나(2,934대/ +495.1%YoY), 싼타페(8,331대/', '+206.1%YoY), 그리고 GV70(3,707대/ +53.6%YoY) 부각. 한편 제네시스는 7,596대(-10.8%MoM, -32.7%YoY).', '15,638대/ 아반테(Elantra) 9,456대, 제네시스 브랜드는 5,724대(+31.5%YoY)로 월 6천 대 수준 하회. 아이오닉 5은 2,979대 / 아이오닉 6은']
        질문 = 팰리세이드는 몇대 판매됐어?? [/INST]
    

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


팰리세이드의 판매량은 주어진 문서에서 언급되지 않습니다. 따라서 내용을 찾을 수 없습니다.</s>


'>'